In [38]:
# Cell 1 — Imports
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [39]:
def preprocess(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [40]:
# Cell 3 — Load Cleaned Dataset
df = pd.read_csv("customer_support_QA_dataset.csv")

df['clean_query'] = df['query'].apply(preprocess)
df['clean_response'] = df['response'].apply(preprocess)

df.head()
df.shape

(1261730, 4)

In [41]:
# Cell 4 — TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=6000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['clean_query'])
X.shape

(1261730, 6000)

# 🤖 Chatbot Retrieval Function

In [42]:
# ---------------------------
# PRO English Translation + Polishing Layer
# ---------------------------
def pro_translate_to_english(text):
    # Basic multilingual stopword detection (PT/ES/FR/DE)
    foreign_chars = r"[à-úãõáéíóúçñäöüßèéêîïôûù]"
    if not re.search(foreign_chars, text.lower()):
        return text  # Already English

    # Larger multilingual dictionary for offline translation
    translate_dict = {
        # Portuguese
        "boa": "good", "bom": "good", "noite": "evening",
        "dia": "day", "tarde": "afternoon", "como": "how",
        "podemos": "we can", "posso": "I can", "ajudar": "help",
        "você": "you", "precisa": "need", "fazer": "do",
        "pagamento": "payment", "problema": "issue",
        "conta": "account", "cliente": "customer",
        "serviço": "service", "mensagem": "message",
        "resolver": "resolve", "pedido": "order",
        "obrigado": "thank you", "desculpe": "sorry",
        "hoje": "today", "conosco": "with us",

        # Spanish
        "buenas": "good", "hola": "hello", "ayuda": "help",
        "pago": "payment", "orden": "order", "soporte": "support",
        "información": "information", "gracias": "thank you",

        # French
        "bonjour": "hello", "merci": "thank you",
        "aide": "help", "problème": "issue", "paiement": "payment",

        # German
        "hilfe": "help", "zahlung": "payment", "konto": "account",
    }

    translated = text

    # Replace foreign words with English words
    for foreign, english in translate_dict.items():
        translated = re.sub(rf"\b{foreign}\b", english, translated, flags=re.IGNORECASE)

    # If text still contains foreign characters → fallback professional English message
    if re.search(foreign_chars, translated.lower()):
        return ("I understand your request. Can you please provide more details "
                "so I can assist you more accurately?")

    # Polish the final output (grammar + clean tone)
    polished = (
        translated.capitalize()
        .replace("  ", " ")
        .replace(" ?", "?")
        .replace(" !", "!")
    )

    # Convert to professional support tone
    return f"{polished}. How else may I assist you today?"

In [43]:
# Remove IDs like @267497
def remove_ids(text):
    return re.sub(r"@\w+", "", text).strip()

# Add brand signature
def add_brand_signature(text):
    return f"{text}\n\n— ProServeAI Support"


In [44]:
def get_response(user_input):
    clean = preprocess(user_input)
    vec = vectorizer.transform([clean])
    sims = cosine_similarity(vec, X).flatten()
    idx = sims.argmax()

    raw_answer = df.iloc[idx]["response"].strip()

    # Low confidence fallback
    if sims[idx] < 0.25:
        return ("I'm not completely certain about that. "
                "Could you please clarify your question?\n\n— ProServeAI Support")

    # Step 1 — Translate automatically to English (PRO)
    english_answer = pro_translate_to_english(raw_answer)

    # Step 2 — Remove IDs like @267497
    english_answer = remove_ids(english_answer)

    # Step 3 — Add Brand Signature
    final_reply = add_brand_signature(english_answer)

    return final_reply


# 🧪 Test the ProServe AI Chatbot

1️⃣**Example - 1**

In [45]:
print(get_response("How do I reset my password?"))

Let's check into that. For guided assistance to reset your password, click here: https://t.co/Le0g5MyEkr

— ProServeAI Support


2️⃣**Example - 2**

In [46]:
print(get_response("I need help with my bill payment"))

We've replied to your DM now, Adam.

— ProServeAI Support


3️⃣**Example - 3**

In [47]:
print(get_response("Can you help me with a refund request?"))

Hi there! We will try our best! Please DM us more info.

— ProServeAI Support


4️⃣**Example - 4**

In [48]:
print(get_response("How do I update my billing address?"))

Great question! Addresses can be changed: https://t.co/NSQ7iYdPbY from within your account settings. ^BK

— ProServeAI Support


5️⃣**Example - 5**

In [49]:
print(get_response("What are your customer support hours?"))

We're closing up shop for the day. But we'll be open normal hours on Friday. -Gabe

— ProServeAI Support


6️⃣**Example - 6**

In [50]:
print(get_response("I have a problem with my order."))

Hey! We're aware of some potential Spotify for Artists log in issues at the moment. The team are on the case 🙂  If you're struggling, can you DM us your account's email? /J https://t.co/ldFdZRiNAt

— ProServeAI Support
